In [18]:
import locator
import pyspark
from pyspark.sql.functions import udf,col, countDistinct
from pyspark.sql.types import StringType,FloatType,DoubleType

#Extra input dataset
file_location = "starbucks_store_locator.csv"
file_type = "csv"
df_starbucks_stores = locator\
    .get_dataset(file_location, file_type, delimiter=",")

# Exploring the dataset
df_starbucks_stores.printSchema()
df_starbucks_stores.show(5)

#Clean dataset.Removed records with null longitudes/latitudes
df_starbucks_stores = df_starbucks_stores\
    .filter((col('Longitude').isNotNull()) | (col('Latitude').isNotNull()))

#Convert lat/long values from string to float for easy computation later on
df_starbucks_stores = df_starbucks_stores\
    .withColumn("Longitude", df_starbucks_stores["Longitude"].cast(FloatType()))\
    .withColumn("Latitude", df_starbucks_stores["Latitude"].cast(FloatType()))

#Augment data with country information
df_country_info = locator.get_country_info()

#Join starbucks data + country info
df_starbucks_country_joined = df_starbucks_stores\
    .join(df_country_info,df_starbucks_stores.Country == df_country_info.ALPHA2,how='left')

df_with_null_countries = df_starbucks_country_joined.filter(col('ALPHA2').isNull())
df_starbucks_stores_with_country_name = df_starbucks_country_joined\
    .filter(col('ALPHA2').isNotNull())


root
 |-- Brand: string (nullable = true)
 |-- Store Number: string (nullable = true)
 |-- Store Name: string (nullable = true)
 |-- Ownership Type: string (nullable = true)
 |-- Street Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- State/Province: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Phone Number: string (nullable = true)
 |-- Timezone: string (nullable = true)
 |-- Longitude: string (nullable = true)
 |-- Latitude: string (nullable = true)

+---------+------------+----------------+--------------+--------------------+----------------+--------------+-------+--------+------------+--------------------+---------+--------+
|    Brand|Store Number|      Store Name|Ownership Type|      Street Address|            City|State/Province|Country|Postcode|Phone Number|            Timezone|Longitude|Latitude|
+---------+------------+----------------+--------------+--------------------+----------------

In [19]:
#1) Which is the most isolated Starbucks store in Australia ?

'''
Assumptions

1. Most Isolated Store/Branch - It means, here which store or branch  has the minimum distance between the next store/branch
   is the largest among its all neighbooring stores/branches.
2. Null valued longitude/latitude will be removed
3. Minimum distance is express in KM
4. Assume Earth is elliptical (Using Haversine distance formula)
'''
import locator
isolated = locator\
    .get_most_isolated_store_per_country(df_starbucks_stores_with_country_name,'AU')
print('The most isolated Starbucks store in Australia is:\n')
print('Store Name: {}\nStore Number: {}\nNearest Branch: {}\nDistance(KM): {}\n'\
      .format(isolated['store_name'], isolated['store_number'],\
       isolated['nearest_store'],isolated['nearest_distance_km']))


The most isolated Starbucks store in Australia is:

Store Name: Mount Druitt
Store Number: 25291-240407
Nearest Branch: Westmead Childrens Hospital
Distance(KM): 16.1



In [20]:
#2) Which country has the least number of Starbucks stores ?

import locator
print ('Countries with least number of Starbucks stores:')
least_stores = locator\
    .get_countries_with_least_starbucks_stores(df_starbucks_stores_with_country_name)
for row in least_stores:
    print ('\nName: {}\nStore Counts: {}\n'.format(row.Country_Name,row.Store_Count))

Countries with least number of Starbucks stores:

Name: Andorra
Store Counts: 1


Name: Niger
Store Counts: 1



In [21]:
#3) How many stores in Sydney has not updated the "Phone Number" in this dataset ?

import locator
df_starbucks_au_old = locator\
    .filter_data(df_starbucks_stores_with_country_name,'Country', 'AU')
df_starbucks_au_old = df_starbucks_au_old.fillna({'Phone Number':''})

#Assume, we will get new file or updated dataset with phone numbers
#In this case, we will just use a replica of the Australian data as an 'updated file'
new_file_location = "au-starbucks.csv"
file_type = "csv"
df_starbucks_au_new = locator.get_dataset(new_file_location, file_type, delimiter=",")
df_starbucks_au_new = locator.filter_data(df_starbucks_au_new, 'Country', 'AU')
df_starbucks_au_new = df_starbucks_au_new.fillna({'Phone Number':''})

#merge two dataset to compare column fields
#First sort both dataset
df_starbucks_au_old_sorted = df_starbucks_au_old\
    .select(df_starbucks_au_old['Store Number']
    ,df_starbucks_au_old['Store Name']\
    ,df_starbucks_au_old['City'].alias('CityA')\
    ,df_starbucks_au_old['Phone Number'].alias('PhoneNumberA'))\
    .orderBy('Store Number')

df_starbucks_au_new_sorted = df_starbucks_au_new\
    .select(df_starbucks_au_new['Store Number']\
    ,df_starbucks_au_new['Store Name']\
    ,df_starbucks_au_new['City'].alias('CityB')\
    ,df_starbucks_au_new['Phone Number'].alias('PhoneNumberB'))\
    .orderBy('Store Number')

#Do the merging here!
df_starbucks_au_merged = locator.merge_dataset(df_starbucks_au_old,\
    df_starbucks_au_new, 'Store Number')

#If PhoneNumberA == PhoneNumberB therefore no changes in phone number  
df_no_phone_numbers_update_sydney = df_starbucks_au_merged\
    .filter((df_starbucks_au_merged['CityA'] == 'Sydney')
    & (df_starbucks_au_merged.PhoneNumberA == df_starbucks_au_merged.PhoneNumberB)) 

print('There are {} stores in Sydney that had not updated the "Phone Number"'\
       .format(df_no_phone_numbers_update_sydney.count()))

df_no_phone_numbers_update_sydney.show()



There are 5 stores in Sydney that had not updated the "Phone Number"
+------------+--------------------+------+------------+------------+--------------------+------+------------+
|Store Number|          Store Name| CityA|PhoneNumberA|Store Number|          Store Name| CityB|PhoneNumberB|
+------------+--------------------+------+------------+------------+--------------------+------+------------+
|25276-240390|           Hyde park|Sydney|            |25276-240390|           Hyde park|Sydney|            |
|25277-240391|           Haymarket|Sydney|            |25277-240391|           Haymarket|Sydney|            |
|25279-240396|   525 George Street|Sydney|            |25279-240396|   525 George Street|Sydney|            |
|25290-240408|Queen Victoria Bu...|Sydney|            |25290-240408|Queen Victoria Bu...|Sydney|            |
|25296-240405|       Wynyard Green|Sydney|            |25296-240405|       Wynyard Green|Sydney|            |
+------------+--------------------+------+---------

In [22]:
#4) Which city has the highest number of "Company Owned" Starbuck stores

#Get a pair-wise frequency table of the columns 'City' and 'Ownership Type'
df_cities_company_owned_stores = locator\
    .get_pair_wise_frequency(df_starbucks_stores_with_country_name\
    ,'City', 'Ownership Type', 'Company Owned')
df_cities_company_owned_stores = df_cities_company_owned_stores\
    .orderBy(col('Company Owned'), ascending=False)
df_cities_company_owned_stores.show(10)
print('There city with highest number of Company Owned Starbuck store is {}'\
    .format(df_cities_company_owned_stores.collect()[0]['City_Ownership Type']))

+-------------------+-------------+
|City_Ownership Type|Company Owned|
+-------------------+-------------+
|             北京市|          234|
|           New York|          224|
|             London|          183|
|            Toronto|          170|
|            Chicago|          127|
|          Singapore|          127|
|             深圳市|          113|
|          Vancouver|          111|
|            Seattle|          108|
|             广州市|          106|
+-------------------+-------------+
only showing top 10 rows

There city with highest number of Company Owned Starbuck store is 北京市
